In [ ]:
import sys
sys.path.append("../TychePlot/")
from PDF import PDF
from Plot_processing import calc, export_data
from SpectraPlot import SpectraPlot
from multiprocessing import Pool
multithreaded=True

In [ ]:
#columns
wavelength=1
intensity_wavelength=2
spectralRadiance_wavelength=3
energy=4
intensity_energy=5
spectralRadiance_energy=6

In [ ]:
#obligatory
#obligatory
title=""
name="X_cspbbr"
fileList=[["X_C_11V_25.1C_54muA.csv"]]
labelList=[r"CsPbBr\textsubscript{3}"]
figscale=0.6
present={#"customFontsize":[8,8,4,6,6],
        "fig_width_pt":665.79451*figscale,
        "scaleX":(665.79451*figscale)/424.75906,
        "fixedFigWidth":True,
        "HWratio":0.82,
        "titleBool":False,
        "filenamePrefix":"presentation_"}
desiredPlots=[
    {
        "xCol":wavelength,
        "xCol2":energy,
        "yCol":intensity_wavelength
    },
    {
        "xCol":wavelength,
        "xCol2":energy,
        "yCol":intensity_wavelength,
        "custom":present
    }
    ]

In [ ]:
optionalParameters={
                    "customLims":False, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#wavelength
                                    None,#spectralRadiance
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#wavelength
                                    None,#spectralRadiance
                                  ],
                    "scaled":False
}

In [ ]:
#optional
figscale=0.6

inputParameters={
                    "customFontsize":[12,12,8,10,10],
                    "labels":labelList,
                    "title":title
                }
inputParametersForScaled=inputParameters.copy()
inputParametersForScaled.update({
                    "scaleX":0.47,
                    "customFontsize":[10,10,4,6,6],
                    "ax2Labels":False,
                    "titleBool":False
                    })

In [ ]:
plots,files=calc(name, fileList, desiredPlots, inputParameters, SpectraPlot, inputParametersForScaled, optionalParameterDict=optionalParameters, multithreaded=multithreaded)

In [ ]:
#export_data(plots)

In [ ]:
PDF(files[0][0], size=(700,700*3/4)) 

In [1]:
if False:
    #FWHM Calc for double Peak
    p=plots[1][0].fitterList[0].params
    covar=plots[1][0].fitterList[0].params_cov
    var=np.diagonal(covar)
    devia=np.sqrt(var)
    FWHM_Red=2*np.sqrt(2*np.log(2))*p[2]
    FWHM_Green=2*np.sqrt(2*np.log(2))*p[5]

    curvDat=plots[1][0].fitterList[0].CurveData
    grad=np.gradient(plots[1][0].fitterList[0].CurveData.getData()[:,1])
    for n in range(0,len(grad)):
        if grad[n]<0:
            break
    first=curvDat.getFirstIndexWhereGreaterOrEq(2,0.5)
    last=curvDat.getLastIndexWhereGreaterOrEq(2,0.5)    
    FWHM_total=curvDat.getSplitData2D()[0][last]-curvDat.getSplitData2D()[0][first]
    maximumPoint=(plots[1][0].fitterList[0].CurveData.getData()[n-1]+plots[1][0].fitterList[0].CurveData.getData()[n-1])/2

    string="Peak{} at {:4.1f} nm / {:4.3f} eV (± {:4.3f} eV) with a FWHM of {:4.3f} eV, that is about {:3.0f} nm"
    print(string.format("_total",np.round(plots[1][0].convFac/maximumPoint[0],decimals=1),np.round(maximumPoint[0],decimals=3),np.round(devia[3],decimals=3),np.round(FWHM_total,decimals=3),np.round(plots[1][0].convFac/(maximumPoint[0]-FWHM_total)-(plots[1][0].convFac/(maximumPoint[0]+FWHM_total)),decimals=0)))
    print(string.format(1,np.round(plots[1][0].convFac/p[0],decimals=1),np.round(p[0],decimals=3),np.round(devia[0],decimals=3),np.round(FWHM_Red,decimals=3),np.round(plots[1][0].convFac/(p[0]-FWHM_Red)-(plots[1][0].convFac/(p[0]+FWHM_Red)),decimals=0)))
    print(string.format(2,np.round(plots[1][0].convFac/p[3],decimals=1),np.round(p[3],decimals=3),np.round(devia[3],decimals=3),np.round(FWHM_Green,decimals=3),np.round(plots[1][0].convFac/(p[3]-FWHM_Green)-(plots[1][0].convFac/(p[3]+FWHM_Green)),decimals=0)))

NameError: name 'plots' is not defined